In [1]:
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
np.set_printoptions(suppress=True)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
trainD = np.load("/home/hsiehch/30s/train_data.npy")
trainL = np.load("/home/hsiehch/30s/train_label.npy")
validationD = np.load("/home/hsiehch/30s/validation_data.npy")
validationL = np.load("/home/hsiehch/30s/validation_label.npy")
testD = np.load("/home/hsiehch/30s/test_data.npy")
testL = np.load("/home/hsiehch/30s/test_label.npy")

trainData = trainD.reshape((trainD.shape[0], trainD.shape[1], 1))
trainLabel = np_utils.to_categorical(trainL, 4)
validationData = validationD.reshape((validationD.shape[0], validationD.shape[1], 1))
validationLabel = np_utils.to_categorical(validationL, 4)
testData = testD.reshape((testD.shape[0], testD.shape[1], 1))
testLabel = np_utils.to_categorical(testL, 4)

print('Train Data:', trainData.shape)
print('Train Label: ', trainLabel.shape)
print('Vali Data: ', validationData.shape)
print('Vali Label: ', validationLabel.shape)
print('Test Data: ', testData.shape)
print('Test Label: ', testLabel.shape)

Train Data: (5078, 9000, 1)
Train Label:  (5078, 4)
Vali Data:  (2032, 9000, 1)
Vali Label:  (2032, 4)
Test Data:  (3041, 9000, 1)
Test Label:  (3041, 4)


In [3]:
graph = tf.Graph()

with graph.as_default():
    inputs = tf.placeholder(tf.float32, shape=(None, 9000, 1), name='inputs')
    labels = tf.placeholder(tf.float32, shape=(None, 4), name='labels')    

    conv1 = tf.layers.conv1d(inputs, filters=32, kernel_size=7, padding='valid', activation=tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(conv1, pool_size=2, strides=2)
    
    conv2 = tf.layers.conv1d(max_pool_1, filters=32, kernel_size=7, padding='valid', activation=tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(conv2, pool_size=2, strides=2)
    
    conv3 = tf.layers.conv1d(max_pool_2, filters=64, kernel_size=7, padding='valid', activation=tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(conv3, pool_size=2, strides=2)
    
    conv4 = tf.layers.conv1d(max_pool_3, filters=64, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(conv4, pool_size=2, strides=2)
    
    conv5 = tf.layers.conv1d(max_pool_4, filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_5 = tf.layers.max_pooling1d(conv5, pool_size=2, strides=2)
    
    conv6 = tf.layers.conv1d(max_pool_5, filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_6 = tf.layers.max_pooling1d(conv6, pool_size=2, strides=2)
    dropout_1 = tf.layers.dropout(max_pool_6, rate=0.5)
    
    conv7 = tf.layers.conv1d(dropout_1, filters=256, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_7 = tf.layers.max_pooling1d(conv7, pool_size=2, strides=2)
    
    conv8 = tf.layers.conv1d(max_pool_7, filters=256, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_8 = tf.layers.max_pooling1d(conv8, pool_size=2, strides=2)
    dropout_2 = tf.layers.dropout(max_pool_8, rate=0.5)
    
    conv9 = tf.layers.conv1d(dropout_2, filters=512, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_9 = tf.layers.max_pooling1d(conv9, pool_size=2, strides=2)
    dropout_3 = tf.layers.dropout(max_pool_9, rate=0.5)
    
    conv10 = tf.layers.conv1d(dropout_3, filters=512, kernel_size=3, padding='valid', activation=tf.nn.relu)
    max_pool_10 = tf.layers.max_pooling1d(conv10, pool_size=2, strides=2)
    
    conv11 = tf.layers.conv1d(max_pool_10, filters=512, kernel_size=3, padding='valid', activation=tf.nn.relu)
    

    
    flatten = tf.layers.flatten(conv11)
    dense_1 = tf.layers.dense(flatten, 128, activation=tf.nn.relu)
    dropout_4 = tf.layers.dropout(dense_1, rate=0.5)
    dense_2 = tf.layers.dense(dropout_4, 32, activation=tf.nn.relu)
    output = tf.layers.dense(dense_2, 4)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=labels))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
 
    # Accuracy
    correct_pred = tf.equal(tf.argmax(output, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [4]:
iteration = 100
batch_size = 70
# batch_index = []
# Make batch index
train_length = trainData.shape[0]
start = 0
end = batch_size

def batch_generation(data_length):
    row = data_length//batch_size
    col = batch_size
    arr = np.arange(row*col).reshape((row, col))
    np.random.shuffle(arr)
    return arr

# while True:    
#     if end + batch_size > train_length:
#         batch_index.append([start, train_length])
#         break
#     else:
#         batch_index.append([start, end])
#         start = end
#         end = end + batch_size

# del start, end

In [5]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    batch_index = batch_generation(trainData.shape[0])
    for epoch in range(iteration):
        for index in batch_index:
            loss, opt, acc = sess.run([cost, optimizer, accuracy],
                             feed_dict={inputs: trainData[index],
                                        labels: trainLabel[index]
                                    })
            
        val_loss, val_acc = sess.run([cost, accuracy],
                             feed_dict={inputs: validationData,
                                        labels: validationLabel
                                    })
        
        print("Epoch: {}/{}".format(epoch+1, iteration))
        print("loss:", "{:09f}".format(loss), "-", "acc:", acc, "-",
              "val_loss:", "{:09f}".format(val_loss), "-", "val_acc:", val_acc)

Epoch: 1/100
loss: 00.352600 - acc: 1.0 - val_loss: 01.008896 - val_acc: 0.5866142
Epoch: 2/100
loss: 00.603331 - acc: 1.0 - val_loss: 00.995078 - val_acc: 0.5866142
Epoch: 3/100
loss: 00.589230 - acc: 1.0 - val_loss: 00.994599 - val_acc: 0.5866142
Epoch: 4/100
loss: 00.611545 - acc: 1.0 - val_loss: 00.995987 - val_acc: 0.5866142
Epoch: 5/100
loss: 00.628307 - acc: 1.0 - val_loss: 00.997708 - val_acc: 0.5866142
Epoch: 6/100
loss: 00.641782 - acc: 1.0 - val_loss: 00.999627 - val_acc: 0.5866142
Epoch: 7/100
loss: 00.645154 - acc: 1.0 - val_loss: 01.000419 - val_acc: 0.5866142
Epoch: 8/100
loss: 00.608382 - acc: 1.0 - val_loss: 00.996233 - val_acc: 0.5866142
Epoch: 9/100
loss: 00.628072 - acc: 1.0 - val_loss: 00.998236 - val_acc: 0.5866142
Epoch: 10/100
loss: 00.581340 - acc: 1.0 - val_loss: 00.994523 - val_acc: 0.5866142
Epoch: 11/100
loss: 00.634050 - acc: 1.0 - val_loss: 00.999104 - val_acc: 0.5866142
Epoch: 12/100
loss: 00.652482 - acc: 1.0 - val_loss: 01.002319 - val_acc: 0.5866142
E

Epoch: 99/100
loss: 00.574091 - acc: 1.0 - val_loss: 00.993757 - val_acc: 0.5866142
Epoch: 100/100
loss: 00.573908 - acc: 1.0 - val_loss: 00.993747 - val_acc: 0.5866142
